In [1]:
import sys
sys.path.append("/Users/zhouzhirui/project/Task-Oriented-Chatbot/")
from pathlib import Path
import re
import gc

import numpy as np
import torch

from chatbot.models.intent.fast_text import FastText
from chatbot.models.intent.text_cnn import TextCNN
from chatbot.models.intent.text_rnn import TextRNN
from chatbot.models.intent.lstm_attention import LSTMAttention
from chatbot.cparse.vocabulary import Vocabulary
from chatbot.cparse.label import Label
from chatbot.models.intent.pytorch import predict, evaluate, batch_generator, train
from chatbot.cparse.text import read_fasttext_file
from chatbot.evaluate.plot import plot_confusion_matrix

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/2z/rt0g_nnn0lb9hrkkzbt5ntr00000gn/T/jieba.cache
Loading model cost 0.784 seconds.
Prefix dict has been built succesfully.


In [2]:
def precessing(s):
    s = re.sub("[.,!?;\\/]", " ", s)
    return s.lower()
x, y = read_fasttext_file("/Users/zhouzhirui/project/Task-Oriented-Chatbot/corpus/intent/fastText/corpus", precessing)
train_x, train_y = x[:8000], y[:8000]
test_x, test_y = x[8000:], y[8000:]
# del x, y; gc.collect()

In [3]:
vocab = Vocabulary()
vocab.update(train_x)
label = Label()
label.update(train_y)

train_x = np.array(vocab.transform(train_x, max_length=50))
test_x = np.array(vocab.transform(test_x, max_length=50))
train_y = np.array(label.transform(train_y))
test_y = np.array(label.transform(test_y))

In [4]:
param_fasttext = {
        "vocab_size": len(vocab),
        "embed_dim": 60,
        "class_num": len(label),
    }

fasttext = FastText(param_fasttext)
print(fasttext)

FastText(
  (embed): Embedding(38326, 60)
  (fc): Linear(in_features=60, out_features=2, bias=True)
)


In [5]:
train(fasttext, train_x, train_y, test_x, test_y, lr=0.01, epochs=10, init_epochs=0, batch_size=64)

2018-05-26 14:59:40,259 - INFO - Intent Model Train - Epoch:  0, Step:      0, train loss: 0.692240, eval loss: 0.691872, acc: 0.516500.
2018-05-26 14:59:42,320 - INFO - Intent Model Train - Epoch:  0, Step:    100, train loss: 0.465118, eval loss: 0.457680, acc: 0.787000.
2018-05-26 14:59:42,857 - INFO - Intent Model Train - Epoch:  1, Step:      0, train loss: 0.296673, eval loss: 0.430793, acc: 0.795500.
2018-05-26 14:59:44,960 - INFO - Intent Model Train - Epoch:  1, Step:    100, train loss: 0.225417, eval loss: 0.410238, acc: 0.815000.
2018-05-26 14:59:45,511 - INFO - Intent Model Train - Epoch:  2, Step:      0, train loss: 0.169546, eval loss: 0.422899, acc: 0.805500.
2018-05-26 14:59:47,676 - INFO - Intent Model Train - Epoch:  2, Step:    100, train loss: 0.114643, eval loss: 0.453374, acc: 0.802500.
2018-05-26 14:59:48,208 - INFO - Intent Model Train - Epoch:  3, Step:      0, train loss: 0.045436, eval loss: 0.461402, acc: 0.804500.
2018-05-26 14:59:50,518 - INFO - Intent M

In [22]:
textcnn_param = {
        "vocab_size": len(vocab),
        "embed_dim": 60,
        "class_num": len(label),
        "kernel_num": 16,
        "kernel_size": [3, 4, 5],
        "dropout": 0.2,
    }

textcnn = TextCNN(textcnn_param)
textcnn

TextCNN(
  (embed): Embedding(38326, 60, padding_idx=1)
  (conv11): Conv2d(1, 16, kernel_size=(3, 60), stride=(1, 1))
  (conv12): Conv2d(1, 16, kernel_size=(4, 60), stride=(1, 1))
  (conv13): Conv2d(1, 16, kernel_size=(5, 60), stride=(1, 1))
  (dropout): Dropout(p=0.2)
  (fc1): Linear(in_features=48, out_features=2, bias=True)
)

In [23]:
train(textcnn, train_x, train_y, test_x, test_y, 0.01, 1, 0, 32)
train(textcnn, train_x, train_y, test_x, test_y, 0.01, 2, 1, 32)
train(textcnn, train_x, train_y, test_x, test_y, 0.003, 6, 2, 128)

2018-05-26 15:13:27,065 - INFO - Intent Model Train - Epoch:  0, Step:      0, train loss: 0.684936, eval loss: 0.711179, acc: 0.502000.
2018-05-26 15:13:31,245 - INFO - Intent Model Train - Epoch:  0, Step:    100, train loss: 0.512509, eval loss: 0.667916, acc: 0.594500.
2018-05-26 15:13:35,354 - INFO - Intent Model Train - Epoch:  0, Step:    200, train loss: 0.475128, eval loss: 0.512748, acc: 0.745000.
2018-05-26 15:13:37,714 - INFO - Intent Model Train - Epoch:  1, Step:      0, train loss: 0.233159, eval loss: 0.744974, acc: 0.636000.
2018-05-26 15:13:42,074 - INFO - Intent Model Train - Epoch:  1, Step:    100, train loss: 0.353971, eval loss: 0.561976, acc: 0.751500.
2018-05-26 15:13:46,189 - INFO - Intent Model Train - Epoch:  1, Step:    200, train loss: 0.297483, eval loss: 0.528387, acc: 0.764000.
2018-05-26 15:13:48,698 - INFO - Intent Model Train - Epoch:  2, Step:      0, train loss: 0.232510, eval loss: 0.506792, acc: 0.767500.
2018-05-26 15:13:55,682 - INFO - Intent M

In [29]:
text_rnn_param = {
    "vocab_size": len(vocab),
    "embed_dim": 60,
    "class_num": len(label),
    "rnn_dim":30,
    "dropout": 0.2,
}

text_rnn = TextRNN(text_rnn_param)
text_rnn

TextRNN(
  (embed): Embedding(38326, 60, padding_idx=0)
  (rnn): LSTM(60, 30, batch_first=True)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=30, out_features=2, bias=True)
)

In [30]:
train(text_rnn, train_x, train_y, test_x, test_y, 0.01, 3, 0, 32)
train(text_rnn, train_x, train_y, test_x, test_y, 0.003, 6, 3, 64)

2018-05-26 15:29:46,862 - INFO - Intent Model Train - Epoch:  0, Step:      0, train loss: 0.725736, eval loss: 0.696585, acc: 0.481500.
2018-05-26 15:29:51,502 - INFO - Intent Model Train - Epoch:  0, Step:    100, train loss: 0.679087, eval loss: 0.683178, acc: 0.556000.
2018-05-26 15:29:55,775 - INFO - Intent Model Train - Epoch:  0, Step:    200, train loss: 0.633901, eval loss: 0.695481, acc: 0.607000.
2018-05-26 15:29:58,121 - INFO - Intent Model Train - Epoch:  1, Step:      0, train loss: 0.506105, eval loss: 0.583320, acc: 0.708500.
2018-05-26 15:30:02,523 - INFO - Intent Model Train - Epoch:  1, Step:    100, train loss: 0.370998, eval loss: 0.527942, acc: 0.752000.
2018-05-26 15:30:06,731 - INFO - Intent Model Train - Epoch:  1, Step:    200, train loss: 0.295909, eval loss: 0.513489, acc: 0.762000.
2018-05-26 15:30:08,949 - INFO - Intent Model Train - Epoch:  2, Step:      0, train loss: 0.306126, eval loss: 0.506561, acc: 0.780500.
2018-05-26 15:30:13,053 - INFO - Intent M

In [4]:
text_rnn_param = {
    "vocab_size": len(vocab),
    "embed_dim": 60,
    "class_num": len(label),
    "rnn_dim":30,
    "dropout": 0.2,
}

lstm_att = LSTMAttention(text_rnn_param)
lstm_att

LSTMAttention(
  (word_embeddings): Embedding(38326, 60)
  (dropout): Dropout(p=0.2)
  (bilstm): LSTM(60, 30, batch_first=True)
  (hidden2label): Linear(in_features=30, out_features=2, bias=True)
)

In [5]:
train(lstm_att, train_x, train_y, test_x, test_y, 0.01, 2, 0, 16)
train(lstm_att, train_x, train_y, test_x, test_y, 0.003, 3, 2, 32)


2018-05-27 14:08:11,307 - INFO - Intent Model Train - Epoch:  0, Step:      0, train loss: 0.680453, eval loss: 0.705631, acc: 0.500000.
2018-05-27 14:08:14,578 - INFO - Intent Model Train - Epoch:  0, Step:    100, train loss: 0.486792, eval loss: 0.645866, acc: 0.634500.
2018-05-27 14:08:17,752 - INFO - Intent Model Train - Epoch:  0, Step:    200, train loss: 0.529697, eval loss: 0.520597, acc: 0.771500.
2018-05-27 14:08:21,049 - INFO - Intent Model Train - Epoch:  0, Step:    300, train loss: 0.290883, eval loss: 0.561221, acc: 0.769000.
2018-05-27 14:08:24,280 - INFO - Intent Model Train - Epoch:  0, Step:    400, train loss: 0.374046, eval loss: 0.474592, acc: 0.781500.
2018-05-27 14:08:27,982 - INFO - Intent Model Train - Epoch:  1, Step:      0, train loss: 0.149938, eval loss: 0.454042, acc: 0.799000.
2018-05-27 14:08:31,581 - INFO - Intent Model Train - Epoch:  1, Step:    100, train loss: 0.241183, eval loss: 0.496505, acc: 0.802000.
2018-05-27 14:08:34,887 - INFO - Intent M

In [30]:
lstm_att.get_attention(torch.tensor(train_x[:2]))[1].squeeze()

tensor([ 9.8122e-06,  9.1842e-04,  3.7952e-04,  1.5840e-03,  2.8121e-03,
         1.4148e-04,  1.1294e-03,  2.4856e-03,  2.2396e-04,  8.3881e-03,
         1.2418e-03,  3.6332e-04,  7.4461e-05,  7.4912e-05,  6.6036e-05,
         2.5978e-04,  6.7352e-05,  6.8104e-05,  5.3283e-03,  5.5602e-02,
         1.4546e-02,  1.5830e-03,  4.0959e-03,  7.4582e-04,  2.2448e-03,
         2.6526e-03,  7.8487e-04,  2.9653e-03,  1.5699e-05,  3.0496e-05,
         1.3183e-04,  3.9311e-04,  8.8378e-04,  6.6267e-04,  2.8145e-04,
         3.2725e-04,  1.1017e-02,  9.7699e-04,  9.0897e-04,  3.0413e-04,
         1.3258e-02,  1.0859e-02,  6.7338e-03,  4.7824e-03,  6.2882e-04,
         4.4202e-02,  4.1129e-03,  5.2195e-02,  2.6186e-02,  7.1027e-01])

In [26]:
" ".join(x[1][:50])

"the best soundtrack ever to anything : i'm reading a lot of reviews saying that this is the best 'game soundtrack' and i figured that i'd write a review to disagree a bit  this in my opinino is yasunori mitsuda's ultimate masterpiece  the music is timeless and i'm"

In [27]:
y[1]

'__label__2'